In [100]:
import pandas as pd
import os
import warnings

In [107]:
fn = os.path.join('futures', 'EOD_Futures_ProductFile_ProductID(578).csv')
df = pd.read_csv(fn, error_bad_lines=False, parse_dates=[0], usecols=[0,1,2,3])

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [118]:
import logging
from datetime import datetime
#from decimal import Decimal
from math import floor

class ConversionAndRounding:
    @staticmethod
    def convertToStrikeForCQGSymbol(barVal = 0, tickIncrementIn = 0, tickDisplayIn = 0, idInstrument = 0):

        if (idInstrument == 39 or idInstrument == 40): # GLE or HE

            return int(barVal * tickDisplayIn)

        elif (idInstrument == 1 or idInstrument == 360):

            return int(barVal * tickDisplayIn)

        else:
            return int(ConversionAndRounding.convertToTickMovesDouble(barVal, tickIncrementIn, tickDisplayIn))

    @staticmethod
    def convertToTickMovesDouble(barVal = 0, tickIncrementIn = 0, tickDisplayIn = 0):
        '''
        Converts double value to nearest tick
        :param barVal:
        :param tickIncrement:
        :param tickDisplay:
        :return:
        '''

        tickIncrement = float(tickIncrementIn)
        tickDisplay = float(tickDisplayIn)

        if tickDisplay == 0:
            return barVal
        try:
            fuzzyZero = tickIncrement / 1000
            positiveFuzzyZero = tickIncrement / 1000

            nTicksInUnit = floor(1 / tickIncrement + positiveFuzzyZero)

            maxFractUnits = (nTicksInUnit - 1) * tickDisplay

            decimalBase = 1
            while (maxFractUnits + positiveFuzzyZero) / decimalBase >= 1:
                decimalBase *= 10

            displayVal = float(barVal)

            print('conversion', displayVal, barVal)

            if displayVal < 0:
                fuzzyZero = -tickIncrement / 1000

            intPart = floor((displayVal + fuzzyZero) / decimalBase + fuzzyZero)

            decPart = (displayVal - intPart * decimalBase) / tickDisplay * tickIncrement

            fractPart = 0

            res = intPart + decPart + fractPart

            incrMultiple = floor(res / tickIncrement + positiveFuzzyZero) * tickIncrement

            if (res < incrMultiple + positiveFuzzyZero and res > incrMultiple - positiveFuzzyZero):
                print('conversion', res)
                return res

            else:
                print('conversion', (incrMultiple + tickIncrement))
                return incrMultiple + tickIncrement;


        except:
            print("conversion error")
            logging.exception("conversion  error")

    @staticmethod
    def convertToStrikeFromSpanData(barVal, strikeIncrement, strikeDisplay, \
                                    idInstrument, currentFileDate):
        '''
        Converts strike to nearest strike increment
        :param barVal:
        :param strikeIncrement:
        :param strikeDisplay:
        :param idInstrument:
        :param currentFileDate:
        :return:
        '''

        try:
            if idInstrument == 1 or idInstrument == 360: #USA=1 ULA=360

                if currentFileDate >= datetime(2016, 3, 7):

                    tempval = float(barVal)
                    tempStrikeDisplay = float(strikeDisplay)

                    return tempval / tempStrikeDisplay

                else:
                    return ConversionAndRounding.convertToTickMovesDoubleSpan(barVal, strikeIncrement, 0)

            elif idInstrument == 532: #LB lumber

                tempval = float(barVal)
                tempStrikeDisplay = float(strikeDisplay)

                return tempval / tempStrikeDisplay

            else:

                return ConversionAndRounding.convertToTickMovesDoubleSpan(barVal, strikeIncrement, strikeDisplay)

        except:
            print("conversion error")
            logging.exception("conversion  error")
            
        
    @staticmethod
    def convertToTickMovesDoubleSpan(barVal = 0, tickIncrementIn = 0, tickDisplayIn = 0):
        '''
        Converts double value to nearest tick
        :param barVal:
        :param tickIncrement:
        :param tickDisplay:
        :return:
        '''

        tickIncrement = float(tickIncrementIn)
        tickDisplay = float(tickDisplayIn)

        if tickDisplay == 0:
            return float(barVal)
        try:
            fuzzyZero = tickIncrement / 1000
            positiveFuzzyZero = tickIncrement / 1000

            nTicksInUnit = floor(1 / tickIncrement + positiveFuzzyZero)

            maxFractUnits = 0
            if nTicksInUnit == 1 and tickDisplay > 0:
                maxFractUnits = nTicksInUnit * tickDisplay
            else:
                maxFractUnits = (nTicksInUnit - 1) * tickDisplay

            decimalBase = 1
            while (maxFractUnits + positiveFuzzyZero) / decimalBase >= 1:
                decimalBase *= 10

            displayVal = float(barVal)

            if displayVal < 0:
                fuzzyZero = -tickIncrement / 1000

            intPart = floor((displayVal + fuzzyZero) / decimalBase + fuzzyZero)

            decPart = (displayVal - intPart * decimalBase) / tickDisplay * tickIncrement

            incrementFixTest = (displayVal - intPart * decimalBase) % (tickIncrement * decimalBase)

            incrementFix = 0

            if incrementFixTest != 0:
                incrementFix = ((tickIncrement * decimalBase) - incrementFixTest) / decimalBase

            return intPart + decPart + incrementFix

        except:
            print("conversion error")
            logging.exception("conversion  error")


In [8]:
def get_month_by_code(month_letter):
    """
    http://www.cmegroup.com/month-codes.html
    January	F
    February	G
    March	H
    April	J
    May	K
    June	M
    July	N
    August	Q
    September	U
    October	V
    November	X
    December	Z
    :param month_letter:
    :return:
    """
    month_letters = {
        'F': 1, 'G': 2, 'H': 3, 'J': 4, 'K': 5, 'M': 6, 'N': 7, 'Q': 8, 'U': 9, 'V': 10, 'X': 11, 'Z': 12}
    return month_letters[month_letter.upper()]

In [15]:
def get_code_by_month(month_year_str):
    month_name = month_year_str[:3]
    month_letters = {
        'Jan':'F',
        'Feb':'G',
        'Mar':'H', 
        'Apr':'J',
        'May':'K', 
        'Jun':'M', 
        'Jul':'N', 
        'Aug':'Q', 
        'Sep':'U', 
        'Oct':'V', 
        'Nov':'X', 
        'Dec':'Z'}
    
    return month_letters[month_name]

In [19]:
def get_year(month_year_str):
    yr = int(month_year_str[-2:])
    if yr > 50:
        return 1900 + yr
    else:
        return 2000 + yr    

In [126]:
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client['tmldb']
exp_collection = db['contractexpirations']
instrument_name = 'CC'
instrument_info = db['instruments'].find_one({'exchangesymbol': instrument_name})

'''
collection.create_index([('idinstrument', pymongo.ASCENDING), 
                         ('optionyear', pymongo.ASCENDING),
                         ('optionmonthint', pymongo.ASCENDING),
                         ('contracttype', pymongo.ASCENDING),
                        ])
'''

def get_expiration_date(df_row, contr_type):
    result = exp_collection.find_one({'idinstrument': instrument_info['idinstrument'],
                                              'optionyear': df_row['year'],
                                              'optionmonthint': df_row['monthint'],
                                              'contracttype': contr_type
                                             })
    if result is None:
        if df_row['year'] > 2012:
            warnings.warn('Expiration not found for\n {0}'.format(df_row))
            
        return None
    else:
        return result['expirationdate']    
    
def get_cqg_name_futures(df_row):
    """
    'self.future_cqg_symbol = SPAN_FILE_CONTRACT_TYPE.future + "." + self.instrument['cqgsymbol'] \
               + self.future_contract_month_char + str(self.future_contract_year % 100)' (edited)
    """
    return 'F.{0}{1}{2}'.format(instrument_info['cqgsymbol'],
                                df_row['month'],
                                df_row['year'] % 100
                               )

def get_contract_name_futures(df_row):
    """
    'self.future_cqg_symbol = SPAN_FILE_CONTRACT_TYPE.future + "." + self.instrument['cqgsymbol'] \
               + self.future_contract_month_char + str(self.future_contract_year % 100)' (edited)
    """
    return 'F.US.{0}{1}{2}'.format(instrument_info['cqgsymbol'],
                                df_row['month'],
                                df_row['year'] % 100
                               )

def get_cqg_name_options(df_row):
    """
    'self.option_cqg_symbol = self.option_type + ".US." + self.instrument['cqgsymbol'] \
                                    + self.option_contract_month_char + str(self.option_contract_year % 100) \
                                    + str(ConversionAndRounding.convertToStrikeForCQGSymbol(self, \
                                                             self.option_strike_price, \
                                                             self.instrument['optionstrikeincrement'], \
                                                             self.instrument['spanstrikedisplay'], \
                                                             self.instrument['idinstrument']))'

    """
    return '{0}.US.{1}{2}{3}{4}'.format(df_row['OptionType'].upper(),
                                instrument_info['cqgsymbol'],
                                df_row['month'],
                                df_row['year'] % 100,
                                ConversionAndRounding.convertToStrikeForCQGSymbol(df_row['StrikePrice'],
                                                             instrument_info['optionstrikeincrement'], 
                                                             instrument_info['spanstrikedisplay'], 
                                                             instrument_info['idinstrument']),
                               )
    


In [127]:
grp = df.groupby(by='MarketID').last().sort_values('Date')

In [128]:
grp['year'] = grp['StripName'].apply(get_year)
grp['month'] = grp['StripName'].apply(get_code_by_month)
grp['monthint'] = grp['month'].apply(get_month_by_code)
grp['expiration'] = grp.apply(get_expiration_date, axis=1, args=(1,))
grp['cqgname'] = grp.apply(get_cqg_name_futures, axis=1)
grp['contractname'] = grp.apply(get_contract_name_futures, axis=1)


In [129]:
grp = grp.dropna()

In [130]:
grp

,Date,ProductName,StripName,year,month,monthint,expiration,cqgname,contractname
MarketID,,,,,,,,,
CC00064,2005-03-16,Cocoa Futures,Mar05,2005,H,3,2005-03-15 00:00:00,F.CCEH5,F.US.CCEH5
CC00065,2005-05-16,Cocoa Futures,May05,2005,K,5,2005-05-13 00:00:00,F.CCEK5,F.US.CCEK5
CC00066,2005-07-15,Cocoa Futures,Jul05,2005,N,7,2005-07-14 00:00:00,F.CCEN5,F.US.CCEN5
CC00067,2005-09-16,Cocoa Futures,Sep05,2005,U,9,2005-09-15 00:00:00,F.CCEU5,F.US.CCEU5
CC00068,2005-12-15,Cocoa Futures,Dec05,2005,Z,12,2005-12-14 00:00:00,F.CCEZ5,F.US.CCEZ5
CC00069,2006-03-17,Cocoa Futures,Mar06,2006,H,3,2006-03-16 00:00:00,F.CCEH6,F.US.CCEH6
CC00070,2006-05-16,Cocoa Futures,May06,2006,K,5,2006-05-15 00:00:00,F.CCEK6,F.US.CCEK6
CC00071,2006-07-17,Cocoa Futures,Jul06,2006,N,7,2006-07-14 00:00:00,F.CCEN6,F.US.CCEN6
CC00072,2006-09-15,Cocoa Futures,Sep06,2006,U,9,2006-09-14 00:00:00,F.CCEU6,F.US.CCEU6


In [131]:
fn_options = os.path.join('options', 'EOD_Options_578_2017.csv')

In [132]:
df_options = pd.read_csv(fn_options, parse_dates=[0])

In [133]:
grp_opt = df_options.groupby(by='OptionMarketID').last().sort_values('Date')
grp_opt['year'] = grp_opt['StripName'].apply(get_year)
grp_opt['month'] = grp_opt['StripName'].apply(get_code_by_month)
grp_opt['monthint'] = grp_opt['month'].apply(get_month_by_code)
grp_opt['expiration'] = grp_opt.apply(get_expiration_date, axis=1, args=(2,))
grp_opt['contractname'] = grp_opt.apply(get_cqg_name_options, axis=1)
grp_opt['cqgname'] = grp_opt.apply(get_cqg_name_options, axis=1)

conversion 3100.0 3100.0
conversion 3100.0
conversion 3050.0 3050.0
conversion 3050.0
conversion 2200.0 2200.0
conversion 2200.0
conversion 2200.0 2200.0
conversion 2200.0
conversion 3000.0 3000.0
conversion 3000.0
conversion 3000.0 3000.0
conversion 3000.0
conversion 2150.0 2150.0
conversion 2150.0
conversion 2150.0 2150.0
conversion 2150.0
conversion 2950.0 2950.0
conversion 2950.0
conversion 2950.0 2950.0
conversion 2950.0
conversion 2100.0 2100.0
conversion 2100.0
conversion 2100.0 2100.0
conversion 2100.0
conversion 2900.0 2900.0
conversion 2900.0
conversion 2900.0 2900.0
conversion 2900.0
conversion 2050.0 2050.0
conversion 2050.0
conversion 2050.0 2050.0
conversion 2050.0
conversion 2850.0 2850.0
conversion 2850.0
conversion 2850.0 2850.0
conversion 2850.0
conversion 2800.0 2800.0
conversion 2800.0
conversion 3250.0 3250.0
conversion 3250.0
conversion 2750.0 2750.0
conversion 2750.0
conversion 2750.0 2750.0
conversion 2750.0
conversion 3050.0 3050.0
conversion 3050.0
conversion 

In [134]:
grp_opt

,Date,TradingType,UnderlyingMarketID,OptionType,StrikePrice,ProductName,ProductID,StripName,Type,FirstPrice,...,HubID,HubName,ClosePrice,OpenInterestChange,year,month,monthint,expiration,contractname,cqgname
OptionMarketID,,,,,,,,,,,,,,,,,,,,,
94372078,2017-01-06,e,5420937,p,3100.0,Cocoa Options,578,Feb17,o,NaN,...,727,NYCC,NaN,0.0,2017,G,2,2017-01-06,P.US.CCEG173100,P.US.CCEG173100
94372082,2017-01-06,e,5420937,p,3050.0,Cocoa Options,578,Feb17,o,NaN,...,727,NYCC,NaN,0.0,2017,G,2,2017-01-06,P.US.CCEG173050,P.US.CCEG173050
94372083,2017-01-06,e,5420937,c,2200.0,Cocoa Options,578,Feb17,o,50.0,...,727,NYCC,50.0,-2.0,2017,G,2,2017-01-06,C.US.CCEG172200,C.US.CCEG172200
94372084,2017-01-06,e,5420937,p,2200.0,Cocoa Options,578,Feb17,o,2.0,...,727,NYCC,1.0,10.0,2017,G,2,2017-01-06,P.US.CCEG172200,P.US.CCEG172200
94372085,2017-01-06,e,5420937,c,3000.0,Cocoa Options,578,Feb17,o,NaN,...,727,NYCC,NaN,0.0,2017,G,2,2017-01-06,C.US.CCEG173000,C.US.CCEG173000
94372086,2017-01-06,e,5420937,p,3000.0,Cocoa Options,578,Feb17,o,NaN,...,727,NYCC,NaN,0.0,2017,G,2,2017-01-06,P.US.CCEG173000,P.US.CCEG173000
94372087,2017-01-06,e,5420937,c,2150.0,Cocoa Options,578,Feb17,o,NaN,...,727,NYCC,NaN,0.0,2017,G,2,2017-01-06,C.US.CCEG172150,C.US.CCEG172150
94372088,2017-01-06,e,5420937,p,2150.0,Cocoa Options,578,Feb17,o,4.0,...,727,NYCC,4.0,10.0,2017,G,2,2017-01-06,P.US.CCEG172150,P.US.CCEG172150
94372089,2017-01-06,e,5420937,c,2950.0,Cocoa Options,578,Feb17,o,NaN,...,727,NYCC,NaN,0.0,2017,G,2,2017-01-06,C.US.CCEG172950,C.US.CCEG172950
